In [1]:
# Install required packages in the current Jupyter kernel
# Uncomment the following lines if you need to install these libraries
# If you run into permission issues, try with the --user option
#import sys
#!pip install -q rdflib networkx matplotlib
#!{sys.executable} -m pip install rdflib networkx matplotlib pandas stringdb --user


In [60]:
%load_ext autoreload
%autoreload 2
import sampler
import grabhogs_sparql

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
datapath = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/402676.protein.links.rdf.v11.5.txt.gz'
# RDF graph loading
rg = sampler.load_graph(datapath)

rdflib Graph loaded successfully with 2620310 triples


In [62]:
subjs = rg.subjects( unique = True)
subg = sampler.sample(rg , subject_iter = subjs , layer_limit= 2 , sample_run = 10)
print(subg)

seed https://string-db.org/network/402676.B6K529
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [63]:
G = rdflib.extras.external_graph_libs.rdflib_to_networkx_multidigraph( subg , lambda s, p, o: {'data':{'key':p  , 'weight':1} } )

In [65]:
datapath2 = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.info.rdf.v11.5/402676.protein.info.rdf.v11.5.txt.gz'
# RDF graph loading
rg_info  = sampler.load_graph(datapath2)
string_ids_dict , subg = sampler.add_xrefs( rg_info , subg )

rdflib Graph loaded successfully with 14559 triples
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [69]:
proteins_by_species , results_subj_para , results_subj_ortho , results_obj_para , results_obj_ortho = grabhogs_sparql.grab_hogs( G , subg , cross_ref)

Total time for 656 SPARQL queries: 190.3214874267578 seconds (multiple batch calls in: 22 cases)
Num errors: 0


In [ ]:
# SPEED UP for cross-refs Uniprot - STRING:
# Using Fuseki server with local TDB2 database for cross-refs only
error_taxa = set()
error_proteins = set()

def map2string_SPARQL(uniprot_id):
    sparql_STRING = SPARQLWrapper("http://dna081:3030/string_fuseki/sparql")
    get_string_id_for_gene = """
    PREFIX lscr: <http://purl.org/lscr#>
    SELECT DISTINCT ?string_id where {
        ?string_id lscr:xrefUniprot \"""" + uniprot_id  + """\"
    } LIMIT 1
    
    """
    
    sparql_STRING.setQuery(get_string_id_for_gene)
    sparql_STRING.setReturnFormat(JSON)
    json_results = sparql_STRING.query().convert()
    if(len(json_results["results"]["bindings"]) > 0):
        return json_results["results"]["bindings"][0]["string_id"]["value"]
    error_proteins.add(uniprot_id)
    return None

results_string_per_protein = {}
num_api_errors = 0

start_time = time.time()
found_in_api = set()

for taxon_id in proteins_by_species.keys():
    for protein in proteins_by_species[taxon_id]:
        try:
            string_id = map2string_SPARQL("http://purl.uniprot.org/uniprot/" + protein)
            if(string_id is not None):
                results_string_per_protein[protein] = string_id
                continue
            # invoking the STRING API is costly, only do this for validation:
            #try:
            #    string_id = stringdb.get_string_ids([protein], int(taxon_id))
            #    if not string_id.empty:
            #        results_string_per_protein[protein] = string_id
            #        found_in_api.add(protein)
            #except Exception:
            #    pass
        except Exception:
            #print(proteins_by_species[taxon_id])
            #print("Error computing IDs for taxon {}".format(taxon_id))
            traceback.print_exc()
            error_taxa.add(taxon_id)
            error_proteins.add(protein)
            num_api_errors += 1
            pass
end_time = time.time()

print("Total time in STRING SPARQL calls: {} seconds. Number of results {} with {} errors".format(end_time - start_time,len(results_string_per_protein), num_api_errors))


In [19]:
# some examples of IDs we can find in the API but not in the RDF

for i, val in enumerate(random.sample(found_in_api, 10)):
    print(val)
    print(results_string_per_protein[val])

/tmp/27457130/ipykernel_1151111/706341601.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for i, val in enumerate(random.sample(found_in_api, 10)):


ValueError: Sample larger than population or is negative

In [ ]:
#error_taxa
#proteins_by_species["10160"]
# sample entry:
for key in results_string_per_protein.keys():
    print(results_string_per_protein[key])
    break

In [ ]:
#sample response

start_time = time.time()
results_obj_para = query_orthologs("<http://purl.uniprot.org/uniprot/B6JXK8>", SPARQLWrapper("https://sparql.omabrowser.org/sparql/"))

end_time = time.time()
print("Total time : {} seconds".format(end_time - start_time))

In [ ]:
# example usage
uniprot_id = "http://purl.uniprot.org/uniprot/A0A3B5R6M3" 
results =  map2string_SPARQL(uniprot_id)
print(results)

In [ ]:
import colour
import itertools

red = colour.Color('red')
blue = colour.Color('blue')

c = [ c.hex_l for c in  list(red.range_to(blue, len(preds))) ]
colors = { p:c[i] for i,p in enumerate(preds)}
delta = 2/len(preds)
curve = { p:delta*i for i,p in enumerate(preds) }
style = itertools.cycle([ '-', '--' ])
line_style = { p:next(style) for p in preds }

In [ ]:
print(colors)

In [ ]:

pos = nx.circular_layout( G )

f = plt.figure()
f.set_figwidth(15)
f.set_figheight(15)
plt.plot()

#plot the whole mess
ax = plt.gca()

for e in G.edges(data = True):
    ax.annotate("",
                xy=pos[e[0]], xycoords='data',
                xytext=pos[e[1]], textcoords='data',
                arrowprops=dict(arrowstyle="-", color=colors[e[2]['data']['key']],
                                shrinkA=5, shrinkB=5, lw = e[2]['data']['weight'], ls = line_style[e[2]['data']['key']],
                                patchA=None, patchB=None, alpha = e[2]['data']['weight'],
                                connectionstyle="arc3,rad=rrr".replace('rrr',str(0.3*curve[e[2]['data']['key']])
                                ),
                                ),
                )

nx.draw_networkx_nodes(G, pos, node_color = 'b', node_size = 150, alpha = .5)
labels=nx.draw_networkx_labels(G , pos = pos , font_size= 15 , font_color='w')

plt.axis('off')
plt.show()


In [ ]:
import gzip
omadata = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/OMA/rdf/oma_hogs_rdf_0.ttl.gz'
with gzip.open(omadata,'rb') as inputdata:
    
    OMArg = RDFGraph()
    OMArg.parse(inputdata.read(), format='turtle')
    print("rdflib Graph loaded successfully with {} triples".format(len(OMArg)))
    

In [ ]:
preds = OMArg.predicates(subject=None, object=None, unique=True)
preds = [ p for  p in preds]
print(preds)

In [ ]:
omadata = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/OMA/rdf/xref_desc.ttl.gz'
with gzip.open(omadata,'rb') as inputdata:
    XREFrg = RDFGraph()
    XREFrg.parse(inputdata.read(), format='turtle')
    print("rdflib Graph loaded successfully with {} triples".format(len(XREFrg)))

In [27]:
#query for hogs here


#check bloom filters for interactors in diff species


#construct networkx graph of the whole subnetwork across species with phylo links for hog



rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Graph loaded successfully with 17287741 triples
rdflib Gra

In [28]:
preds = XREFrg.predicates(subject=None, object=None, unique=True)
preds = [ p for  p in preds]
print(preds)

[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment')]


In [15]:
import pickle 
import glob

filters = './filters/bloomfinal_big*.pkl'
filters = glob.glob(filters)
filters = [ pickle.load(open(f, 'rb')) for f in filters ]
print(filters)

[[(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 76358434)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 80606663)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 71174921)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 77390257)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 70656044)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 78053087)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 69379149)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 63716625)], [(BloomFilter(ideal_num_elements_n=100000000, error_rate_p=0.001000, num_bits_m=1437758757), 75075008)], [(BloomFilter(ideal_num_elements_n=100000000, error_ra

In [16]:
def check_filters(val,filters = filters):
    for f in filters:
        if val in f[0]:
            return True
    return False
print(check_filters('crap'))
print(check_filters('23.BEL05_00185 _23.BEL05_00025 '.encode()))
print(check_filters('23.BEL05_00045 23.BEL05_02665'))

False
False
False


In [17]:
import dask
from dask.distributed import fire_and_forget
from dask.distributed import Client, Variable , Queue , Lock ,LocalCluster
from dask_jobqueue import SLURMCluster
from dask.distributed import  utils_perf
from dask.distributed import Client, LocalCluster
import dask
import redis
from bloom_filter2 import BloomFilter
import lzma
from dask import dataframe as dd
path = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/'
link_df = dd.read_csv(path + 'datasets/STRING/protein.links.full.v11.5.txt',  blocksize=15e6 , header = 0, sep = ' ')
print(link_df)
sampledf = link_df.sample(frac = .0000000001).compute()
print(sampledf)

Dask DataFrame Structure:
                   protein1 protein2 neighborhood neighborhood_transferred fusion cooccurence homology coexpression coexpression_transferred experiments experiments_transferred database database_transferred textmining textmining_transferred combined_score
npartitions=102701                                                                                                                                                                                                                                             
                     object   object        int64                    int64  int64       int64    int64        int64                    int64       int64                   int64    int64                int64      int64                  int64          int64
                        ...      ...          ...                      ...    ...         ...      ...          ...                      ...         ...                     ...      ...                  ...

/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python3.10/site-packages/dask/dataframe/io/csv.py:129: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(

KeyboardInterrupt



In [18]:
sample = 1000
data ={}
import pandas as pd
path = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/'
with open(path + 'datasets/STRING/protein.links.full.v11.5.txt' ) as stringlinks:
    for i,line in enumerate( stringlinks):
        if i > sample:
            break
        if i > 0:
            data[i]= {j:w for j,w in enumerate(line.split())}
sampledf = pd.DataFrame.from_dict(data, orient = 'index')
print(sampledf)

                  0               1  2  3  4    5  6  7    8  9  10 11 12 13  \
1     23.BEL05_00185  23.BEL05_00025  0  0  0  169  0  0    0  0  0  0  0  0   
2     23.BEL05_00025  23.BEL05_00390  0  0  0  155  0  0    0  0  0  0  0  0   
3     23.BEL05_00390  23.BEL05_00025  0  0  0  155  0  0    0  0  0  0  0  0   
4     23.BEL05_00025  23.BEL05_00435  0  0  0  196  0  0  174  0  0  0  0  0   
5     23.BEL05_00435  23.BEL05_00025  0  0  0  196  0  0  174  0  0  0  0  0   
...              ...             ... .. .. ..  ... .. ..  ... .. .. .. .. ..   
996   23.BEL05_00035  23.BEL05_01175  0  0  0  150  0  0    0  0  0  0  0  0   
997   23.BEL05_01175  23.BEL05_00035  0  0  0  150  0  0    0  0  0  0  0  0   
998   23.BEL05_00035  23.BEL05_01195  0  0  0    0  0  0    0  0  0  0  0  0   
999   23.BEL05_01195  23.BEL05_00035  0  0  0    0  0  0    0  0  0  0  0  0   
1000  23.BEL05_00035  23.BEL05_01235  0  0  0  154  0  0    0  0  0  0  0  0   

       14   15  
1       0  169  
2    

In [19]:
def add_links(df):
    df = df[~df[0].isna()]
    df = df[~df[1].isna()]
    df.protein1 = df[0].map(lambda x:str(x))
    df.protein2 = df[1].map(lambda x:str(x))
    df['protlinks'] = df[0] + '_' + df[1] 
    return df
sampledf = add_links(sampledf)
print(sampledf)

                   0               1  2  3  4    5  6  7    8  9 10 11 12 13  \
1     23.BEL05_00185  23.BEL05_00025  0  0  0  169  0  0    0  0  0  0  0  0   
2     23.BEL05_00025  23.BEL05_00390  0  0  0  155  0  0    0  0  0  0  0  0   
3     23.BEL05_00390  23.BEL05_00025  0  0  0  155  0  0    0  0  0  0  0  0   
4     23.BEL05_00025  23.BEL05_00435  0  0  0  196  0  0  174  0  0  0  0  0   
5     23.BEL05_00435  23.BEL05_00025  0  0  0  196  0  0  174  0  0  0  0  0   
...              ...             ... .. .. ..  ... .. ..  ... .. .. .. .. ..   
996   23.BEL05_00035  23.BEL05_01175  0  0  0  150  0  0    0  0  0  0  0  0   
997   23.BEL05_01175  23.BEL05_00035  0  0  0  150  0  0    0  0  0  0  0  0   
998   23.BEL05_00035  23.BEL05_01195  0  0  0    0  0  0    0  0  0  0  0  0   
999   23.BEL05_01195  23.BEL05_00035  0  0  0    0  0  0    0  0  0  0  0  0   
1000  23.BEL05_00035  23.BEL05_01235  0  0  0  154  0  0    0  0  0  0  0  0   

       14   15                      pro

/tmp/25549704/ipykernel_2112516/3218239898.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.protein1 = df[0].map(lambda x:str(x))
/tmp/25549704/ipykernel_2112516/3218239898.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.protein2 = df[1].map(lambda x:str(x))


In [21]:
#check sampledf in filters
sampledf['check'] = sampledf.protlinks.map( lambda x : 1 if check_filters(x) == True else 0 )
print(sampledf.check.sum())

0


In [29]:


#sample string subnet

#get OMA HOGs

#jump to a few close species 

#

#collapse nodes with degree 2


In [30]:
i = 0 
from rdflib.namespace import RDF

for s, p, o in rg.triples((None,  RDF.type, None)):
    print(f"{s} is a {o}")
    if i > 10:
        break

In [ ]:
#map string neighbours to OMA entries

#jump a few steps in HOGs

#fish for STRING interactions in other species


